https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE228026
    
tissue: triple-negative breast cancer
cell line: MDA-MB-231
    

Cells were treated with DMSO control (samples 1-3) or UF010 (10 µM, samples 4-6) for 24 hours. Samples were treated with DMSO (samples 7-9, and 19-21), YX968 (50 nM, samples 10-12, and 22-24), YX968-NC (samples 13-15, and 25-27), or YXYX26862 (samples 16-18, and 28-30) for 6h (samples 7-18) or 24h (samples 19-30).

## Loading

In [1]:
library("limma")
library("edgeR")
library("readr")
library(biomaRt)
library(dplyr)   



Attaching package: 'dplyr'


The following object is masked from 'package:biomaRt':

    select


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
library("depmap")
library("ExperimentHub")
library("dplyr")
library("ggplot2")
library("stringr")

snapshotDate(): 2021-10-19

This is depmap release 21Q3

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:dplyr':

    combine, intersect, setdiff, union


The following object is masked from 'package:limma':

    plotMA


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: AnnotationHub

Loading required package: BiocFileCache

Loading required package: dbplyr


Attaching package: 'dbplyr'


The following objec

In [3]:
crispr <- depmap_crispr()
rnai <- depmap_rnai()
drug_sensitivity <- depmap_drug_sensitivity()
RPPA <- depmap_RPPA()
TPM <- depmap_TPM()
metadata <- depmap_metadata()
mutationCalls <- depmap_mutationCalls()
proteomic <- depmap_proteomic()


snapshotDate(): 2021-10-19

see ?depmap and browseVignettes('depmap') for documentation

loading from cache

snapshotDate(): 2021-10-19

see ?depmap and browseVignettes('depmap') for documentation

loading from cache

snapshotDate(): 2021-10-19

see ?depmap and browseVignettes('depmap') for documentation

loading from cache

snapshotDate(): 2021-10-19

see ?depmap and browseVignettes('depmap') for documentation

loading from cache

snapshotDate(): 2021-10-19

see ?depmap and browseVignettes('depmap') for documentation

loading from cache

snapshotDate(): 2021-10-19

see ?depmap and browseVignettes('depmap') for documentation

loading from cache

snapshotDate(): 2021-10-19

see ?depmap and browseVignettes('depmap') for documentation

loading from cache

snapshotDate(): 2021-10-19

see ?depmap and browseVignettes('depmap') for documentation

loading from cache



In [4]:
## R & edgeR versions 
sprintf("R version used for this notebook is : %s %s", R.Version()$major, R.Version()$minor)
sprintf("edgeR used for this notebook is : %s ", packageVersion('edgeR'))

[1] "R version used for this notebook is : 4 1.3"

[1] "edgeR used for this notebook is : 3.36.0 "

In [5]:
datatyp <- "UF010"
cellline <- "MDAMB231_BREAST"
celllinename <-'MDAMB231_BREAST'
resultfilename <- paste(datatyp, "results.csv", sep = "")
filename <- paste(datatyp, ".tsv", sep = "")
outputfilename <- paste(datatyp, "_merged.csv", sep = "")

resultfilename
filename
outputfilename

[1] "UF010results.csv"

[1] "UF010.tsv"

[1] "UF010_merged.csv"

### <b> Read RNA-Seq Data</b>


In [175]:
result_data <- read_csv(resultfilename,show_col_types = FALSE)
dim(result_data)
head(result_data,2)


[1] 29592     2

ID,up_down_regulated
<dbl>,<dbl>
16657436,0
16657440,0


In [176]:
org_data <- read_tsv(filename,show_col_types = FALSE)
org_data <- merge(org_data, result_data, by = "ID")
dim(org_data)
head(org_data,2)

[1] 29592    13

,ID,adj.P.Val,P.Value,t,B,logFC,RANGE_STRAND,RANGE_START,RANGE_END,GB_ACC,SPOT_ID,RANGE_GB,up_down_regulated
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,16657436,0.0593,0.023,2.594729,-4.420923,0.3347525,+,12190,13639,NR_046018,chr1:12190-13639,NC_000001.10,0
2,16657440,0.3890,0.257,1.187869,-6.590851,0.1376707,+,29554,31109,NA,chr1:29554-31109,NC_000001.10,0


In [177]:

head(org_data,2)



,ID,adj.P.Val,P.Value,t,B,logFC,RANGE_STRAND,RANGE_START,RANGE_END,GB_ACC,SPOT_ID,RANGE_GB,up_down_regulated
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,16657436,0.0593,0.023,2.594729,-4.420923,0.3347525,+,12190,13639,NR_046018,chr1:12190-13639,NC_000001.10,0
2,16657440,0.3890,0.257,1.187869,-6.590851,0.1376707,+,29554,31109,NA,chr1:29554-31109,NC_000001.10,0


### <b> Find the genes</b>


In [145]:
mart <- useMart("ensembl")
datasets <- listDatasets(mart)
mart <- useDataset("hsapiens_gene_ensembl",mart)


In [197]:
att <- listAttributes(mart)
attcon <- grep("seq", att$name, value=TRUE)
attcon


[1] "refseq_mrna"              "refseq_mrna_predicted"   
[3] "refseq_ncrna"             "refseq_ncrna_predicted"  
[5] "refseq_peptide"           "refseq_peptide_predicted"

In [198]:
getBM(attributes=c("ensembl_gene_id"),
      filters="hgnc_symbol", values="AAMDC", mart=mart)


ensembl_gene_id
<chr>
ENSG00000087884


In [146]:
getBM(attributes=c("affy_hugene_1_0_st_v1","affy_hugene_2_0_st_v1"),
      filters="hgnc_symbol", values="AAMDC", mart=mart)


affy_hugene_1_0_st_v1,affy_hugene_2_0_st_v1
<int>,<int>
7942783,NA
NA,16729444
NA,NA
7942783,16729444


In [202]:
att <- listAttributes(mart)
#attcon <- grep("hgnc", att$name, value=TRUE)
#attcon
write.csv(att,'listAttributes.csv')

In [ ]:
chromosome_name: Knowing the chromosome location can be informative in the context of gene regulation and potential interactions with neighboring genes.
start_position and end_position: These define the location of the gene on the chromosome, which might be relevant for understanding its regulatory environment.
strand: This indicates whether the gene is located on the positive or negative strand of the DNA, which can influence transcription.
transcript_start and transcript_end: These define the boundaries of the transcribed RNA molecule, which is crucial for understanding gene expression.
transcript_length: The length of the transcript can be informative, as longer transcripts might be less efficiently translated.
external_gene_name: This provides a human-readable name for the gene, which can be helpful for interpretation.
gene_biotype: This

In [207]:
BiocManager::install("ChIPseeker")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.14 (BiocManager 1.30.22), R 4.1.3 (2022-03-10)

Installing package(s) 'ChIPseeker'

also installing the dependencies 'caTools', 'KernSmooth', 'boot', 'gplots', 'gtools', 'plotrix', 'TxDb.Hsapiens.UCSC.hg19.knownGene'





  There are binary versions available but the source versions are later:
             binary  source needs_compilation
KernSmooth  2.23-20 2.23-24              TRUE
boot       1.3-28.1  1.3-30             FALSE
gplots        3.1.3 3.1.3.1             FALSE
gtools        3.9.4   3.9.5              TRUE
plotrix       3.8-2   3.8-4             FALSE

  Binaries will be installed
package 'caTools' successfully unpacked and MD5 sums checked
package 'KernSmooth' successfully unpacked and MD5 sums checked
package 'gtools' successfully unpacked and MD5 sums checked
package 'ChIPseeker' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\1G6652897\AppData\Local\Temp\RtmpI5bldY\downloaded_packages


installing the source packages 'boot', 'gplots', 'plotrix', 'TxDb.Hsapiens.UCSC.hg19.knownGene'


Old packages: 'ape', 'askpass', 'backports', 'BH', 'BiocManager', 'brew',
  'brio', 'broom', 'bslib', 'cachem', 'Cairo', 'callr', 'cli', 'commonmark',
  'credentials', 'crosstalk', 'curl', 'data.table', 'DBI', 'dbplyr', 'desc',
  'digest', 'dplyr', 'DT', 'evaluate', 'fansi', 'farver', 'fastmap',
  'fastmatch', 'filelock', 'fs', 'gert', 'ggforce', 'ggfun', 'ggplot2',
  'ggraph', 'ggrepel', 'glue', 'gprofiler2', 'graphlayouts', 'gtable',
  'gtools', 'haven', 'highr', 'htmltools', 'htmlwidgets', 'httpuv', 'igraph',
  'jsonlite', 'KernSmooth', 'knitr', 'later', 'lattice', 'lifecycle', 'locfit',
  'lubridate', 'matrixStats', 'mgcv', 'munsell', 'nlme', 'openssl', 'pbdZMQ',
  'pkgload', 'plotly', 'plyr', 'polyclip', 'prettyunits', 'processx',
  'profvis', 'progress', 'promises', 'ps', 'purrr', 'ragg', 'Rcpp',
  'RcppArmadillo', 'RcppEigen', 'RCurl', 'readr', 'readxl', 'remotes', 'repr',
  'reticu

ERROR: Error in parse(text = x, srcfile = src): <text>:2:8: unexpected symbol
1: # Import libraries
2: import requests
          ^


In [188]:
getBM(attributes=c("hgnc_symbol","hgnc_trans_name","hgnc_id","affy_hugene_2_0_st_v1"),
      filters="affy_hugene_2_0_st_v1", values="17118403", mart=mart)


hgnc_symbol,hgnc_trans_name,hgnc_id,affy_hugene_2_0_st_v1
<lgl>,<lgl>,<lgl>,<int>
NA,NA,NA,17118403


In [147]:
gene_id_ds <- getBM(attributes=c("hgnc_symbol","affy_hugene_2_0_st_v1"), mart=mart)
dim(gene_id_ds)
head(gene_id_ds,2)

[1] 75859     2

,hgnc_symbol,affy_hugene_2_0_st_v1
,<chr>,<int>
1,MT-TF,17100639
2,MT-TF,17100641


In [148]:
gene_id_ds <- rename(gene_id_ds, gene_name = hgnc_symbol)
gene_id_ds <- rename(gene_id_ds, ID = affy_hugene_2_0_st_v1)
dim(gene_id_ds)
head(gene_id_ds,2)

[1] 75859     2

,gene_name,ID
,<chr>,<int>
1,MT-TF,17100639
2,MT-TF,17100641


In [199]:
getSequence(id = "BRCA1", 
                  type = "hgnc_symbol", 
                  seqType = "peptide", 
                  mart = mart)

peptide,hgnc_symbol
<chr>,<chr>
MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEFCNKSKQPGLARSQHNRWAGSKETCNDRRTPSTEKKVDLNADPLCERKEWNKQKLPCSENPRDTEDVPWITLNSSIQKVNEWFSRSDELLGSDDSHDGESESNAKVADVLDVLNEVDEYSGSSEKIDLLASDPHEALICKSERVHSKSVESNIEDKIFGKTYRKKASLPNLSHVTENLIIGAFVTEPQIIQERPLTNKLKRKRRPTSGLHPEDFIKKADLAVQKTPEMINQGTNQTEQNGQVMNITNSGHENKTKGDSIQNEKNPNPIESLEKESAFKTKAEPISSSISNMELELNIHNSKAPKKNRLRRKSSTRHIHALELVVSRNLSPPNCTELQIDSCSSSEEIKKKKYNQMPVRHSRNLQLMEGKEPATGAKKSNKPNEQTSKRHDSDTFPELKLTNAPGSFTKCSNTSELKEFVNPSLPREEKEEKLETVKVSNNAEDPKDLMLSGERVLQTERSVESSSISLVPGTDYGTQESISLLEVSTLGKAKTEPNKCVSQCAAFENPKGLIHGCSKDNRNDTEGFKYPLGHEVNHSRETSIEMEESELDAQYLQNTFKVSKRQSFAPFSNPGNAEEECATFSAHSGSLKKQSPKVTFECEQKEENQGKNESNIKPVQTVNITAGFPVVGQKDKPVDNAKCSIKGGSRFCLSSQFRGNETGLITPNKHGLLQNPYRIPPLFPIKSFVKTKCKKNLLEENFEEHSMSPEREMGNENIPSTVSTISRNNIRENVFKEASSSNINEVGSSTNEVGSSINEIGSSDENIQAELGRNRGPKLNAMLRLGVLQPEVYKQSLPGSNCKHPEIKKQEYEEVVQTVNTDFSPYLISDNLEQPMGSSHASQVCSETPDDLLDDGEIKEDTSFAENDIKESSAVFSKSVQKGELSRSPSPFTHTHLAQGYRRGAKKLESSEENLSSEDEELPCFQHLLFGKVNNIPSQSTRHSTVATECLSKNTEENLLSLKNSLNDCSNQVILAKASQEHHLSEETKCSASLFSSQCSELEDLTANTNTQDPFLIGSSKQMRHQSESQGVGLSDKELVSDDEERGTGLEENNQEEQSMDSNLGEAASGCESETSVSEDCSGLSSQSDILTTQQRDTMQHNLIKLQQEMAELEAVLEQHGSQPSNSYPSIISDSSALEDLRNPEQSTSEKVLTSQKSSEYPISQNPEGLSADKFEVSADSSTSKNKEPGVERSSPSKCPSLDDRWYMHSCSGSLQNRNYPSQEELIKVVDVEEQQLEESGPHDLTETSYLPRQDLEGTPYLESGISLFSDDPESDPSEDRAPESARVGNIPSSTSALKVPQLKVAESAQSPAAAHTTDTAGYNAMEESVSREKPELTASTERVNKRMSMVVSGLTPEEFMLVYKFARKHHITLTNLITEETTHVVMKTDAEFVCERTLKYFLGIAGGKWVVSYFWVTQSIKERKMLNEHDFEVRGDVVNGRNHQGPKRARESQDRKIFRGLEICCYGPFTNMPTDQLEWMVQLCGASVVKELSSFTLGTGVHPIVVVQPDAWTEDNGFHAIGQMCEAPVVTREWVLDSVALYQCQELDTYLIPQIPHSHY*,BRCA1
MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGEAASGCESETSVSEDCSGLSSQSDILTTQQRDTMQHNLIKLQQEMAELEAVLEQHGSQPSNSYPSIISDSSALEDLRNPEQSTSEKVLTSQKSSEYPISQNPEGLSADKFEVSADSSTSKNKEPGVERSSPSKCPSLDDRWYMHSCSGSLQNRNYPSQEELIKVVDVEEQQLEESGPHDLTETSYLPRQDLEGTPYLESGISLFSDDPESDPSEDRAPESARVGNIPSSTSALKVPQLKVAESAQSPAAAHTTDTAGYNAMEESVSREKPELTASTERVNKRMSMVVSGLTPEEFMLVYKFARKHHITLTNLITEETTHVVMKTDAEFVCERTLKYFLGIAGGKWVVSYFWVTQSIKERKMLNEHDFEVRGDVVNGRNHQGPKRARESQDRKIFRGLEICCYGPFTNMPTGCPPNCGCAARCLDRGQWLPCNWADV*,BRCA1
MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPL*,BRCA1
Sequence unavailable,BRCA1
MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEFCNKSKQPGLARSQHNRWAGSKETCNDRRTPSTEKKVDLNADPLCERKEWNKQKLPCSENPRDTEDVPWITLNSSIQKVNEWFSRSDELLGSDDSHDGESESNAKVADVLDVLNEVDEYSGSSEKIDLLASDPHEALICKSERVHSKSVESNIEDKIFGKTYRKKASLPNLSHVTENLIIGAFVTEPQIIQERPLTNKLKRKRRPTSGLHPEDFIKKADLAVQKTPEMINQGTNQTEQNGQVMNITNSGHENKTKGDSIQNEKNPNPIESLEKESAFKTKAEPISSSISNMELELNIHNSKAPKKNRLRRKSSTRHIHALELVVSRNLSPPNCTELQIDSCSSSEEIKKKKYNQMPVRHSRNLQLMEGKEPATGAKKSNKPNEQTSKRHDSDTFPELKLTNAPGSFTKCSNTSELKEFVNPSLPREEKEEKLETVKVSNNAEDPKDLMLSGERVLQTERSVESSSISLVPGTDYGTQESISLLEVSTLGKAKTEPNKCVSQCAAFENPKGLIHGCSKDNRNDTEGFKYPLGHEVNHSRETSIEMEESELDAQYLQNTFKVSKRQSFAPFSNPGNAEEECATFSAHSGSLKKQSPKVTFECEQKEENQGKNESNIKPVQTVNITAGFPVVGQKDKPVDNAKCSIKGGSRFCLSSQFRGNETGLITPNKHGLLQNPYRIPPLFPIKSFVKTKCKKNLLEENFEEHSMSPEREMGNENIPSTVSTISRNNIRENVFKEASSSNINEVGSSTNEVGSSINEIGSSDENIQAELGRNRGPKLNAMLRLGVLQPEVYKQSLPGSNCKHPEIKKQEYEEVVQTVNTDFSPYLISDNLEQPMGSSHASQVCSETPDDLLDDGEIKEDTSFAENDIKESSAVFSKSVQKGELSRSPSPFTHTHLAQGYRRGAKKLESSEENLSSEDEELPCFQHLLFGKVNNIPSQSTRHSTVATECLSKNTEENLLSLKNSLNDCSNQVILAKASQEHHLSEETKCSA

In [178]:
regulated_geneds <- merge(org_data, gene_id_ds, by = "ID")
dim(regulated_geneds)
head(regulated_geneds,2)

[1] 31823    14

,ID,adj.P.Val,P.Value,t,B,logFC,RANGE_STRAND,RANGE_START,RANGE_END,GB_ACC,SPOT_ID,RANGE_GB,up_down_regulated,gene_name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,16657436,0.0593,0.023,2.594729,-4.420923,0.3347525,+,12190,13639,NR_046018,chr1:12190-13639,NC_000001.10,0,DDX11L17
2,16657436,0.0593,0.023,2.594729,-4.420923,0.3347525,+,12190,13639,NR_046018,chr1:12190-13639,NC_000001.10,0,


In [179]:
sum(is.na(regulated_geneds$gene_name))


[1] 0

## Now All the other data

In [151]:
#1. Crispr dependency
crispr_mdaamb231 <- crispr %>%
dplyr::select(cell_line, gene_name, dependency) %>%
dplyr::filter(stringr::str_detect(cell_line, cellline)) %>%
dplyr::arrange(dependency)
crispr_mdaamb231 <- rename(crispr_mdaamb231, crispr_dependency = dependency)

dim(crispr_mdaamb231)
head(crispr_mdaamb231,2)


[1] 17386     3

cell_line,gene_name,crispr_dependency
<chr>,<chr>,<dbl>
HS695T_SKIN,RPS29,-2.703321
HS695T_SKIN,PSMB3,-2.699420


In [152]:
#2. RNAI dependency
rnai_mdaamb231 <- rnai %>%
dplyr::select(cell_line, gene_name, dependency) %>%
dplyr::filter(stringr::str_detect(cell_line, celllinename)) %>%
dplyr::arrange(dependency)
rnai_mdaamb231 <- rename(rnai_mdaamb231, rnai_dependency = dependency)
dim(rnai_mdaamb231)
head(rnai_mdaamb231,2)

[1] 17309     3

cell_line,gene_name,rnai_dependency
<chr>,<chr>,<dbl>
HS695T_SKIN,SFPQ,-2.366918
HS695T_SKIN,PSMA3,-2.341507


In [153]:
#3. Avg RNA Exp

mdaamb231_TPM <- TPM %>%
  filter(cell_line == celllinename)

avg_rna_exp <- mdaamb231_TPM %>%
  group_by(gene_name) %>%
  summarise(average_rna_expression = mean(rna_expression, na.rm = TRUE))
dim(avg_rna_exp)
head(avg_rna_exp,2)


[1] 19177     2

gene_name,average_rna_expression
<chr>,<dbl>
A1BG,4.46008724
A1CF,0.01435529


In [154]:
#4. Average protein exp
mdaamb231_proteomic <- proteomic %>%
  filter(cell_line == celllinename)
avg_protein_exp <- mdaamb231_proteomic %>%
  group_by(gene_name) %>%
  summarise(average_protein_expression = mean(protein_expression, na.rm = TRUE))

dim(avg_protein_exp)
head(avg_protein_exp,2)

[1] 12197     2

gene_name,average_protein_expression
<chr>,<dbl>
A1CF,-0.8639875
A2M,0.6545716


In [ ]:
seqType

In [155]:
# 5 Count of transcripted genes:
gene_transcript <- getBM(attributes=c("hgnc_symbol","transcript_start", "transcript_end"),  mart=mart)
gene_transcript_sorted <- gene_transcript[order(gene_transcript$hgnc_symbol), ]

count_gene_transcript <- gene_transcript_sorted %>%
  group_by(hgnc_symbol) %>%
    summarise(cnt_gene_transcript = n())
count_gene_transcript <- rename(count_gene_transcript, gene_name = hgnc_symbol)

dim(count_gene_transcript)
head(count_gene_transcript)
count_gene_transcript <- subset(count_gene_transcript, !is.na(gene_name) & gene_name != "")

dim(count_gene_transcript)
head(count_gene_transcript)

[1] 40983     2

gene_name,cnt_gene_transcript
<chr>,<int>
,36157
A1BG,5
A1BG-AS1,9
A1CF,9
A2M,13
A2M-AS1,4


[1] 40982     2

gene_name,cnt_gene_transcript
<chr>,<int>
A1BG,5
A1BG-AS1,9
A1CF,9
A2M,13
A2M-AS1,4
A2ML1,9


In [156]:
# 6  Gene location, size
gene_locs <- getBM(attributes=c("hgnc_symbol", "chromosome_name",
                                "start_position","end_position"),  mart=mart)
gene_locs$gene_length=gene_locs$end_position - gene_locs$start_position
gene_locs <- rename(gene_locs, gene_name = hgnc_symbol)
dim(gene_locs)
head(gene_locs,2)


[1] 70609     5

,gene_name,chromosome_name,start_position,end_position,gene_length
,<chr>,<chr>,<int>,<int>,<int>
1,MT-TF,MT,577,647,70
2,MT-RNR1,MT,648,1601,953


unique(gene_locs$transcript_biotype)
transcript_biotype:
    'Mt_tRNA''Mt_rRNA''protein_coding''snRNA''rRNA''misc_RNA''pseudogene''processed_pseudogene''retained_intron''protein_coding_CDS_not_defined''lncRNA''TEC''miRNA''nonsense_mediated_decay''protein_coding_LoF''unprocessed_pseudogene''transcribed_unprocessed_pseudogene''transcribed_processed_pseudogene''rRNA_pseudogene''snoRNA''non_stop_decay''transcribed_unitary_pseudogene''IG_V_gene''IG_V_pseudogene''unitary_pseudogene''IG_C_pseudogene''sRNA''TR_J_gene''TR_V_gene''TR_J_pseudogene''TR_D_gene''TR_C_gene''IG_D_gene''scaRNA''ribozyme''artifact''IG_C_gene''processed_transcript''IG_J_gene''IG_J_pseudogene''translated_processed_pseudogene''TR_V_pseudogene''IG_pseudogene''scRNA''vault_RNA'


In [157]:
# get all protein coding, RNA genes:

proteintypes=c("protein_coding","protein_coding_CDS_not_defined","protein_coding_LoF")
rnatypes=c("Mt_tRNA","Mt_rRNA","snRNA","rRNA","misc_RNA","lncRNA",
           "miRNA","rRNA_pseudogene","snoRNA","sRNA","scaRNA","scRNA","vault_RNA")

protein_genes <- getBM(attributes=c("hgnc_symbol"),
      filters="transcript_biotype", values=proteintypes, mart=mart)

rna_genes <- getBM(attributes=c("hgnc_symbol"),
      filters="transcript_biotype", values=rnatypes, mart=mart)


In [158]:
protein_genes <- rename(protein_genes, gene_name = hgnc_symbol)
rna_genes <- rename(rna_genes, gene_name = hgnc_symbol)

dim(protein_genes)
head(protein_genes,2)
dim(rna_genes)
head(rna_genes,2)

[1] 19424     1

,gene_name
,<chr>
1,MT-ND1
2,MT-ND2


[1] 11446     1

,gene_name
,<chr>
1,MT-TF
2,MT-RNR1


In [159]:
regulated_geneds$is_protein_gene <- ifelse(regulated_geneds$gene_name %in% protein_genes$gene_name, 1, 0)
regulated_geneds$is_rna_genes <- ifelse(regulated_geneds$gene_name %in% rna_genes$gene_name, 1, 0)
dim(regulated_geneds)
head(regulated_geneds,2)

[1] 31823    16

,ID,adj.P.Val,P.Value,t,B,logFC,RANGE_STRAND,RANGE_START,RANGE_END,GB_ACC,SPOT_ID,RANGE_GB,up_down_regulated,gene_name,is_protein_gene,is_rna_genes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,16657436,0.0593,0.023,2.594729,-4.420923,0.3347525,+,12190,13639,NR_046018,chr1:12190-13639,NC_000001.10,0,DDX11L17,0,0
2,16657436,0.0593,0.023,2.594729,-4.420923,0.3347525,+,12190,13639,NR_046018,chr1:12190-13639,NC_000001.10,0,,0,0


In [160]:
dim(gene_locs)

[1] 70609     5

In [182]:
### Now merge
dim(regulated_geneds)

[1] 31823    15

In [183]:
regulated_geneds$is_crispr <- ifelse(regulated_geneds$gene_name %in% crispr_mdaamb231$gene_name, 1, 0)


merged1 <- dplyr::left_join(regulated_geneds, crispr_mdaamb231,
                                    by = c("gene_name"))  
merged1 <- dplyr::left_join(merged1, rnai_mdaamb231,
                                    by = c("gene_name")) 
merged1 <- dplyr::left_join(merged1, avg_rna_exp,
                                    by = c("gene_name")) 
#merged1 <- dplyr::full_join(merged1, avg_protein_exp,      by = c("gene_name")) 
merged_final <- dplyr::left_join(merged1, count_gene_transcript,
                                    by = c("gene_name")) 

dim(merged_final) 


[1] 31823    21

In [169]:
merged1 <- merge(regulated_geneds, crispr_mdaamb231, by = "gene_name")
merged2 <- merge(merged1, rnai_mdaamb231, by = "gene_name")
merged3 <- merge(merged2, avg_rna_exp, by = "gene_name")
#merged3 <- merge(merged3, avg_protein_exp, by = "gene_name")
merged_final <- merge(merged3, count_gene_transcript, by = "gene_name")
dim(merged_final)

[1] 14649    22

In [170]:
merged_final <- rename(merged_final, PValue = P.Value)


In [184]:
write.csv(merged_final, outputfilename)